# General input output format

In [ ]:
def tokenize_sentece(str):
    # S="I met David Beckham and Victoria in Madrid"
    # Solr = "I met *David Beckham* and *Victoria* in *Madrid*"
    # S = [I,met, David Beckham, .... ] using string.punctuation + white_spaces
    # M=[2,4,5]
    returns S,M

In [ ]:
def generate_candidates(S, max_t=10, enforce=True):
    """ Given a sentence list (S) and  a mentions list (M), returns a list of candiates
        Inputs:
            S: segmented sentence [w1, ..., wn]
            M: mensions [m1, ... , mj]
            max_t: maximum candiate per mention
            enforce: Makes sure the "correct" entity is among the candidates
        Outputs:
         Candidate list [[(c11, p11),...(c1k, p1k)],...[(cn1, pn1),...(c1m, p1m)]]
             where cij is the jth candidate for ith mention and pij is the relative frequency of cij
    
    """
    candslist=[]
    for m in M:
#         wid = title2id(m[1])
#         if wid is None:
#             raise Exception(m[1].encode('utf-8') + ' not found')
        
        clist = anchor2concept(S[m[0]])
        clist = sorted(clist, key=lambda x: -x[1])

        smooth=0    
        trg = [(i,(c,f)) for i,(c,f) in enumerate(clist) if c==wid]
        if enforce and (not trg):
            trg=[(len(clist), (wid,0))]
            smooth=1

            
        clist = clist[:max_t]
        if enforce and (smooth==1 or trg[0][0]>=max_t): 
            if clist:
                clist.pop()
            clist.append(trg[0][1])
        s = sum(c[1]+smooth for c in clist )        
        clist = [(c,float(f+smooth)/s) for c,f in clist ]
            
        candslist.append(clist)
    return  candslist 

In [ ]:
import sys
sys.path.insert(0,'..')

from wsd.wsd import *

S=["Carlos", "met", "David", "and" , "Victoria", "in", "Madrid"]
M=[[2, "David_Beckham"], [4, "Victoria_Beckham"], [6, "Madrid"]]

C=generate_candidates(S, M, max_t=3, enforce=False)
def get_popularity(C):
    scores=[]
    for c in C:
        s=[x[1] for x in c]
        scores.append(s)
    return scores
        
P=get_popularity(C)    

# WORD2VEC

In [5]:
import sys
sys.path.insert(0,'..')
from wikisim.calcsim import *
word2vec = gensim_loadmodel('/home/sajadi/backup/wikipedia/WikipediaClean5Negative300Skip10.Ehsan/WikipediaClean5Negative300Skip10')

/home/sajadi/anaconda2/lib/python2.7/site-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


In [7]:
x=getentity2vector(encode_entity('David_Beckham','word2vec_id'))
y=getentity2vector(encode_entity('David', 'word2vec_id'))
context=getword2vector('soccer') + getword2vector('Victoria')


In [8]:
import scipy as sp
import scipy.sparse as sprs
import scipy.spatial
import scipy.sparse.linalg 
#panda vector
print 1-sp.spatial.distance.cosine(context, x);
print 1-sp.spatial.distance.cosine(context, y);

0.369183122053
0.0909524586892


In [10]:
getword2vector('soccer')

0     -0.286691
1      0.309643
2     -0.440182
3     -0.215264
4      0.363785
5     -0.298642
6      0.574758
7     -0.094513
8      0.318274
9     -0.188464
10     0.158765
11    -0.257204
12     0.237741
13    -0.257524
14     0.288563
15     0.545386
16    -0.393122
17    -0.317044
18    -0.169808
19     0.098242
20    -0.258380
21    -0.262747
22    -0.194904
23    -0.444793
24     0.277448
25    -0.025654
26     0.287121
27     0.086031
28    -0.122469
29    -0.239167
         ...   
270   -0.426469
271    0.274891
272   -0.258263
273   -0.112344
274   -0.116418
275    0.355874
276    0.386146
277    0.044444
278   -0.362351
279   -0.217038
280    0.320433
281   -0.161443
282   -0.083336
283    0.226705
284    0.167725
285    0.398921
286   -0.117666
287    0.395689
288   -0.112931
289   -0.145321
290   -0.136716
291   -0.071161
292   -0.309932
293    0.302323
294    0.217616
295    0.093228
296   -0.085171
297    0.438386
298   -0.120025
299    0.221915
dtype: float32

In [3]:
from gensim import *
from embeddings import *

ImportError: No module named embeddings